In [1]:
import os
import sys
sys.path.append("..")
from src.embed import STextEmbedder
import pandas as pd
import weaviate
import weaviate.classes.config as wc


In [2]:
# create a collection locally with 
client = weaviate.connect_to_local()
embedder = STextEmbedder()

In [9]:
client.collections.delete("hnsl_vec_db")

In [10]:
client.collections.create(
    name="hnsl_vec_db",
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT),
        wc.Property(name="url", data_type=wc.DataType.TEXT),
        wc.Property(name="chunk_uuid", data_type=wc.DataType.UUID)
    ]
)

In [11]:
collection = client.collections.get("hnsl_vec_db")

In [7]:
df = pd.read_csv("../yamrzou_links.csv")

In [8]:
from uuid import uuid4
for index, row in df.iterrows():
    filename = row['title'].lower().replace(" ", "_")
    filename = filename.replace(".", "_")
    try:
        with open(f"../contents/{filename}.txt", "r") as f:
            corpus = f.read()
        embeddings = embedder.embed(corpus)
        for embedding in embeddings:
            chunk_uuid = uuid4()
            collection.data.insert(
            properties={
                "Title": row['title'],
                "url": row['url'],
                "chunk_uuid": chunk_uuid
            },
            vector=embedding.tolist(),
        )
        if index % 100 == 0:
            print(f"Finished indexing URL {index} / {len(df)}")    
    except Exception as e:
        pass

[Errno 2] No such file or directory: '../contents/the_art_and_science_of_mess_management_(1981)_[pdf].txt'
[Errno 2] No such file or directory: '../contents/meaningful_(2019).txt'
[Errno 2] No such file or directory: '../contents/slothful_induction_and_ad_hoc_escapism_(2009).txt'
[Errno 2] No such file or directory: '../contents/the_importance_of_goal_disengagement_in_adaptive_self-regulation_(2003)_[pdf].txt'
[Errno 2] No such file or directory: '../contents/why_making_friends_as_an_adult_is_harder.txt'
[Errno 2] No such file or directory: '../contents/tv_as_mediator_of_community_(2010).txt'
[Errno 2] No such file or directory: '../contents/all_that_is_solid_melts_into_information.txt'
[Errno 2] No such file or directory: '../contents/chronic_pain_is_surprisingly_treatable_–_when_patients_focus_on_the_brain_(2021).txt'
[Errno 2] No such file or directory: '../contents/epistemic_akrasia_(2013)_[pdf].txt'
[Errno 2] No such file or directory: '../contents/new_rules_applying_for_eu_crypto

/opt/anaconda3/lib/python3.12/site-packages/executing/executing.py:713: DeprecationWarning: ast.Str is deprecated and will be removed in Python 3.14; use ast.Constant instead
  right=ast.Str(s=sentinel),
/opt/anaconda3/lib/python3.12/ast.py:587: DeprecationWarning: Attribute s is deprecated and will be removed in Python 3.14; use value instead
  return Constant(*args, **kwargs)


KeyboardInterrupt: 